# Title Thesis defense

In [1]:
import locale
import os
import re
import time
from typing import List, Tuple

from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose


locale.setlocale(locale.LC_ALL, '');

## Loading M4 Data

In [2]:
def read_m4_data(file_paths: List[str], path_prefix: str) -> pd.DataFrame:
    """ reads the  list given file paths and
        and combines them into a singular dataframe"""
    df_all = pd.DataFrame()
    
    for fpath in file_paths:
        start = time.time()
        df_tmp = pd.read_csv(path_prefix+fpath)
        end = time.time()
        execution_time = end - start
        print(f"file: {fpath} read in {execution_time:.2f} seconds\n\
        with {df_tmp.shape[0]:,d} rows and {df_tmp.shape[1]:,d} columns.\n")
        df_all = pd.concat([df_all, df_tmp])
        
    return df_all

In [3]:
# setting file paths
path_prefix_m4 = "/Users/philipp/workspace/UNIC/comp-593/m4_data/"
m4_paths = [
    "Hourly-train.csv",
#     "Daily-train.csv",
    "Weekly-train.csv",
    # "Monthly-train.csv",
    # "Quarterly-train.csv",
    # "Yearly-train.csv"
]

In [4]:
%%time
df_all_m4 = read_m4_data(m4_paths, path_prefix_m4)
print("###################\n")

file: Hourly-train.csv read in 0.12 seconds
        with 414 rows and 961 columns.

file: Weekly-train.csv read in 0.32 seconds
        with 359 rows and 2,598 columns.

###################

CPU times: user 399 ms, sys: 45.9 ms, total: 445 ms
Wall time: 457 ms


### Load converted M4 time series

In [5]:
def extend_missing_freqs(freqs_l: list) -> list:
    # missing frequencies indicator = -1
    
    freqs_ext = [-1]*(5-len(freqs_l))
    freqs_ext.extend(freqs_l)


    return freqs_ext

In [6]:
df_m4_repo = pd.read_csv("../data/df_stats.csv",
                        converters = {"freq_ids": lambda x: eval(x)})
df_m4_repo["freq_ids"] = df_m4_repo["freq_ids"].apply(extend_missing_freqs)


In [7]:
ls = oth = 0
for i, r in df_m4_repo.iterrows():
    if len(r["freq_ids"])==5:
        ls+=1
    else:
        oth+=1
print(f"ls: {ls}\nother: {oth}")

ls: 300000
other: 0


## Load UCR archive

In [8]:
def get_ucr_file_paths(path_prefix: str) -> Tuple[List[str],List[str]]:
  """creates a list file paths based on naming
  conventions of UCR archive"""
  ts_train_infos = []
  ts_test_infos = []
  for root, dirs, files in os.walk(path_prefix):
      for name in files:
          if(name.endswith("_TRAIN.tsv")):
              path_tmp = os.path.join(root,name)
              ts_name = re.split("/", root)[-1]
              ts_train_infos.append((ts_name, os.path.join(root,name)))
          elif(name.endswith("_TEST.tsv")):
              path_tmp = os.path.join(root,name)
              ts_name = re.split("/", root)[-1]
              ts_test_infos.append((ts_name, os.path.join(root,name)))
  return ts_train_infos, ts_test_infos

In [9]:
path_prefix_ucr = "/Users/philipp/workspace/UNIC/comp-593/data/ucr_data/UCRArchive_2018"
ts_train_paths, ts_test_paths = get_ucr_file_paths(path_prefix_ucr)

In [10]:
def load_ucr_files(train_paths: List[str], test_paths: List[str]) -> Tuple[pd.DataFrame,pd.DataFrame]:
    """load UCR archive files"""
    df_train = pd.DataFrame()
    df_test = pd.DataFrame()

    for ts_info in tqdm(train_paths):
        ts_name = ts_info[0]
        fp = ts_info[1]

        df_tmp = pd.read_csv(fp, sep='\t', header=None)
        df_tmp['name'] = ts_name
        df_tmp['no'] = df_tmp.index
        cols = df_tmp.columns.tolist()
        cols = cols[-2:] + cols[:-2]
        df_tmp = df_tmp[cols]
        df_train = df_train.append(df_tmp)

    for ts_info in tqdm(test_paths):
        ts_name = ts_info[0]
        fp = ts_info[1]

        df_tmp = pd.read_csv(fp, sep='\t', header=None)
        df_tmp['name'] = ts_name
        df_tmp['no'] = df_tmp.index
        cols = df_tmp.columns.tolist()
        cols = cols[-2:] + cols[:-2]
        df_tmp = df_tmp[cols]
        df_test = df_test.append(df_tmp)
        
    return df_train, df_test
    

In [11]:
df_train, df_test = load_ucr_files(ts_train_paths, ts_test_paths)

  0%|          | 0/143 [00:00<?, ?it/s]

  0%|          | 0/143 [00:00<?, ?it/s]

## Visualize Time Series

In [12]:
sns.set_style("darkgrid", {"grid.color": ".6", "grid.linestyle": ":"})

## Load FFT conversion results with statistics

In [13]:
# load m4 stats
df_m4_stats = pd.read_csv("../data/df_stats.csv")

# load UCR stats
#df_ucr_stats_train = pd.read_csv("../data/df_ucr_stats_train.csv")
df_ucr_stats_test = pd.read_csv("../data/df_ucr_stats_test.csv")

### Sample Output M4 statsfile
please note identified frequency bands and computed statistics


In [14]:
df_m4_stats[df_m4_stats['ts_name']=='M1']

,ts_name,freq_ids,type,m,b,count,mean,std,min,q25,q50,q75,max
104641,M1,"[35, 0, 12, 25, 0]",Welch,0.763289,6125.215235,469.0,6306.247335,1790.611195,2690.0,5000.0,6040.0,7360.0,13490.0
144641,M1,"[409, 198, 136, 409, 0]",fft,0.763289,6125.215235,469.0,6306.247335,1790.611195,2690.0,5000.0,6040.0,7360.0,13490.0
244641,M1,"[409, 198, 409, 136, 0]",Hamming,0.763289,6125.215235,469.0,6306.247335,1790.611195,2690.0,5000.0,6040.0,7360.0,13490.0


### Sample Ouput UCR statsfile

In [15]:
df_ucr_stats_test.head()

,ts_name,freq_ids,type,no,class,m,b,count,mean,std,min,q25,q50,q75,max
0,ACSF1,"[340, 388, 370, 340, 388]",Hamming,0.0,9.0,1460,-2.927752e-06,0.002134,-1.541096e-09,1.0,-0.577967,-0.577967,-0.577967,0.012759,1.742434
1,ACSF1,"[340, 388, 370, 340, 388]",Hamming,1.0,9.0,1460,3.756977e-06,-0.002682,5.684931e-10,1.0,-0.598240,-0.588575,-0.588332,0.027480,1.756899
2,ACSF1,"[388, 340, 370, 388, 340]",Hamming,2.0,9.0,1460,-2.508553e-06,0.001803,-2.739726e-10,1.0,-0.586960,-0.582897,-0.582691,0.013297,1.757700
3,ACSF1,"[340, 388, 370, 388, 340]",Hamming,3.0,9.0,1460,-9.970090e-06,0.007221,9.041096e-10,1.0,-0.591978,-0.590736,-0.583757,0.032882,1.746551
4,ACSF1,"[340, 388, 370, 388, 340]",Hamming,4.0,9.0,1460,3.325016e-07,-0.000261,4.315069e-10,1.0,-0.577828,-0.577828,-0.577828,0.008326,1.743008


## Transform Frequencies M4

In [16]:
def get_top_k_freq(PSD: np.array, k: int)->List[int]:
    """ return top k indexes with largest PSD val"""
    PSD = [np.real(val) for val in PSD]
    return sorted(range(len(PSD)), key= lambda x: PSD[x])[-k:]

In [17]:
def get_freq_m4(s: pd.Series, k:int=5) -> List[float]:
    """ compute frequencies for M4 pandas series"""
    df = pd.DataFrame()
    f=np.array(s.iloc[1:].dropna())

    n = f.size
    wdw = np.hamming(n)
    freq = np.arange(n)/n

    # FFT
    fhat = np.abs(np.fft.fft(f))
    PSD = np.real(fhat * np.conj(fhat) / n)
    top_fft_idx = get_top_k_freq(PSD,k)
    fft_freq = freq[top_fft_idx]
    print(f" {s.iloc[0]} fft freqs: {fft_freq}")
    df_tmp = pd.DataFrame(fft_freq, columns=['val'])
    df_tmp['type']='fft'
    df = df.append(df_tmp)
    # Hamming
    fhat = np.abs(np.fft.fft(f*wdw))
    PSD = np.real(fhat * np.conj(fhat) / n)
    freq = np.arange(n)/f.size
    top_ham_idx = get_top_k_freq(PSD,k)
    ham_freq = freq[top_ham_idx]
    df_tmp = pd.DataFrame(ham_freq, columns=['val'])
    df_tmp['type']='Hamming'
    df = df.append(df_tmp)
    # Welch
    seg_length = np.floor(1/20*n)
    if seg_length == 0:
        seg_length=10
    welch_freqs, PSD_welch = signal.welch(f, nperseg=seg_length,
                                      window='hamming')
    top_ham_idx = get_top_k_freq(PSD_welch,k)
    welch_freq = freq[top_ham_idx]
    df_tmp = pd.DataFrame(welch_freq, columns=['val'])
    df_tmp['type']='Welch'
    df = df.append(df_tmp)
    
    return df

## Transform to frequencies UCR

In [18]:
def get_freq_ucr(s: pd.Series, k:int=5) -> List[float]:
    """ compute frequencies for M4 pandas series"""
    df = pd.DataFrame()
    f=np.array(s.iloc[3:].dropna())
    n = f.size
    wdw = np.hamming(n)
    freq = np.arange(n)/n

    # FFT
    fhat = np.fft.fft(f)
    PSD = np.real(fhat * np.conj(fhat) / n)
    top_fft_idx = get_top_k_freq(PSD,k)
    fft_freq = freq[top_fft_idx]
    df_tmp = pd.DataFrame(fft_freq, columns=['val'])
    df_tmp['type']='fft'
    df = df.append(df_tmp)
    # Hamming
    fhat = np.fft.fft(f*wdw)
    PSD = np.real(fhat * np.conj(fhat) / n)
    freq = np.arange(n)/f.size
    top_ham_idx = get_top_k_freq(PSD,k)
    ham_freq = freq[top_ham_idx]
    df_tmp = pd.DataFrame(ham_freq, columns=['val'])
    df_tmp['type']='Hamming'
    df = df.append(df_tmp)
    # Welch
    seg_length = np.floor(1/20*n)
    if seg_length == 0:
        seg_length=10
    welch_freqs, PSD_welch = signal.welch(f, nperseg=seg_length,
                                      window='hamming')
    top_ham_idx = get_top_k_freq(PSD_welch,k)
    welch_freq = freq[top_ham_idx]
    df_tmp = pd.DataFrame(welch_freq, columns=['val'])
    df_tmp['type']='Welch'
    df = df.append(df_tmp)
    return df

### Test on Single Series

### set frequency ranges
build a set of frequency ranges $ [10^{-4}, ..., 10^1] $ with a 0.01 step-size

In [19]:
start = -4
stop = 1
steps = 501
freq_ranges = 10**np.linspace(start,stop,steps)

### Convert frequencies to frequency ranges and add statistics

In [20]:
def top_freqs_2_idx(s: pd.DataFrame) -> pd.Series:
    """convert the list of frequencies into order"""
    ar_top_f = np.asarray(s.transpose().iloc[0,:].tolist())
    ar_f_idx = np.digitize(ar_top_f, freq_ranges)
    #print(f"type: {str(s.iloc[0,1])}\nfreqs: {top_f}\n\n")
    df_res = pd.DataFrame.from_dict({"fft_type": [str(s.iloc[0,1])],
                 "top_freq": [ar_f_idx]})
    return df_res

In [21]:
def get_trend(ar:np.ndarray, period:int=12)->Tuple[float,float]:
    """ time series decomposition """
    periodicity=12
    if ar.shape[0] < 2*periodicity:
        periodicity = math.floor(ar.shape[0]/2)
    
    res = seasonal_decompose(ar, 'additive', period=periodicity)
    return res.trend[~np.isnan(res.trend)]


def fit_trend(trend_ar: np.ndarray) -> Tuple[float,float]:
    fit_res = np.polyfit(np.arange(trend_ar.shape[0]), trend_ar, 1)
    m = fit_res[0]
    b = fit_res[1]
    return (m,b)

In [22]:
def get_statistics_m4(s:pd.Series
                     )->pd.DataFrame:
    """compute simple statistics on time series"""
    ts_name = s[0]
    ar = np.array(s.iloc[1:].dropna())
    
    count = ar.shape[0]
    mean = np.mean(ar)
    std = np.std(ar)
    min_val = np.min(ar)
    q25 = np.quantile(ar, .25)
    q50 = np.median(ar)
    q75 = np.quantile(ar, .75)
    max_val = np.max(ar)
    
    period = 12
    trend_ar = get_trend(ar, period)
    m, b = fit_trend(trend_ar)

    idx = ['ts_name', 'm', 'b', 'count', 'mean', 'std', 'min', 'q25', 'q50', 'q75', 'max']
    res = pd.Series([ts_name, m, b, count, mean, std, min_val, q25, q50, q75, max_val], index=idx)
    return res

### Convert example times series in M4 to for mapping to time series search engine database

In [40]:
#%%timeit

# get one random time series
df_test = df_all_m4.sample(1, random_state=1).squeeze().dropna()

# identify the associated frequencies of time series
df_test_freq = get_freq_m4(df_test)
# compute statistics on time series
test_stats = get_statistics_m4(df_test)
print(test_stats)

# order frequencies according to strength of signal
df_test_conv = df_test_freq.groupby(['type']).apply(top_freqs_2_idx)

#set series name
df_test_conv["ts_name"] =df_test[0]

# create row vectored  stats dataframe
test_stats = pd.DataFrame(test_stats).T

# index by time series name
test_stats.index = test_stats["ts_name"]

# join dataframes with frequencies (by different windows) and join with stats
df_test_conv = df_test_conv.set_index("ts_name").join(test_stats)

 W184 fft freqs: [9.98783455e-01 1.21654501e-03 6.08272506e-04 9.99391727e-01
 0.00000000e+00]
ts_name           W184
m             2.593652
b           -96.688459
count             1644
mean       2022.421086
std        1257.281934
min           527.3252
q25           744.0792
q50         1837.06745
q75        3279.209725
max          4624.0424
dtype: object


### Identify closest series via TS Search Algorithm

## Frequency matching algorithm

#### this method handles the frequency comparison

In [24]:
def compare_freqs(df_sub: pd.DataFrame,
    freq_l: list) -> pd.Series:
    """compute match score
        df_sub -> contains template candidates
        freq_l -> list of frequencies of template
    """
   
    # convert frequencies column to array
    cand_l = list(df_sub['freq_ids'])
    candidates = np.array(cand_l)
    
    template_ar = np.array([freq_l]*candidates.shape[0])

    # match the frequencies
    #print(f"cand: {candidates[:3]}\n{np.array(df_sub['freq_ids'].tolist())[:2]}")
    match_ar =  candidates-template_ar

    mask = match_ar == 0

    indx = np.nonzero(mask)

    """
     on the matches apply 10 to power of column index
     and set non-matches to zero
    """
    match_ar[mask] = 10**(indx[1])
    match_ar[~mask] = 0

    match_scores = pd.Series(np.sum(match_ar, axis=1))
    return match_scores

In [32]:
df_test_conv

,fft_type,top_freq,ts_name,m,b,count,mean,std,min,q25,q50,q75,max
ts_name,,,,,,,,,,,,,
W184,Hamming,"[109, 400, 79, 400, 0]",W184,2.593652,-96.688459,1644,2022.421086,1257.281934,527.3252,744.0792,1837.06745,3279.209725,4624.0424
W184,Welch,"[139, 127, 0, 109, 79]",W184,2.593652,-96.688459,1644,2022.421086,1257.281934,527.3252,744.0792,1837.06745,3279.209725,4624.0424
W184,fft,"[400, 109, 79, 400, 0]",W184,2.593652,-96.688459,1644,2022.421086,1257.281934,527.3252,744.0792,1837.06745,3279.209725,4624.0424


In [33]:
df_m4_repo[df_m4_repo["ts_name"]=="W184"]

,ts_name,freq_ids,type,m,b,count,mean,std,min,q25,q50,q75,max
36735,W184,"[409, 111, 81, 409, 0]",fft,2.603752,-65.327517,1644.0,2022.421086,1257.664494,527.3252,744.0792,1837.06745,3279.209725,4624.0424
76735,W184,"[111, 409, 81, 409, 0]",Hamming,2.603752,-65.327517,1644.0,2022.421086,1257.664494,527.3252,744.0792,1837.06745,3279.209725,4624.0424
236735,W184,"[0, 0, 0, 0, 0]",Welch,2.603752,-65.327517,1644.0,2022.421086,1257.664494,527.3252,744.0792,1837.06745,3279.209725,4624.0424


In [29]:
df_match_scores = generate_stats(df_test_conv,
                                 df_m4_repo)
#df_match_scores.head()

No match found for:
W254 - Hamming
No match found for:
W254 - Welch
No match found for:
W254 - fft


#### this method creates comparison dataframe

In [28]:
def generate_stats(df_templ: pd.Series,
                  df_m4_repo: pd.Series) -> pd.DataFrame:
    """
    compute match coefficient for each column
    """
    df_res = pd.DataFrame()
    for idx, row in df_templ.iterrows():
        ts_name_1 = idx
    #     no_1 = stats_ar[1]
    #     cls_type = stats_ar[2]
        freq_l = row["top_freq"]
        fft_type = row["fft_type"]
        m = row["m"]
        b = row["b"]
        count = row["count"]
        mean = row["mean"]
        std = row["std"]
        min_val = row["min"]
        q25 = row["q25"]
        q50 = row["q50"]
        q75 = row["q75"]
        max_val = row["max"]
    
        df_sub = df_m4_repo[df_m4_repo["type"]==fft_type]
        df_sub.reset_index(inplace=True)
        #df_sub = get_global_df(fft_type)
        # remove unnecessary candidates from df_g
        if m>0:
            df_sub = df_sub[(df_sub['m']>0)]
        elif m==0:
            raise Exception("handle special case of exact stationarity")
        else:
            df_sub = df_sub[(df_sub['m']<0)]
            

        cols = ['ts_1', 'ts_2', 'type', 'match_score', 'd_m',
                'd_mean', 'd_std', 'd_count', 'd_q25', 'd_q50', 'd_q75']
        df_tmp = pd.DataFrame(columns=cols)
        df_tmp['ts_1'] = [ts_name_1]*df_sub.shape[0]
        #df_tmp['no_1'] = no_1
        #df_res['class_1'] = cls_type
        df_tmp['ts_2'] = df_sub['ts_name'].reset_index(drop=True)
        #df_res['no_2'] = df_sub['no'].reset_index(drop=True)
        #df_res['class_2'] = df_sub['class'].reset_index(drop=True)
        df_tmp['type'] = [fft_type]*df_sub.shape[0]
    
  
        match_scores = compare_freqs(df_sub, freq_l)
        if match_scores.shape[0] == df_sub.shape[0]:
            df_res['match_score'] = match_scores
        else:
            df_res['match_score'] = match_scores

        # compute delta statistics
        df_tmp['d_m'] = df_sub['m'].apply(lambda x: abs(x-m)).reset_index(drop=True)
        df_tmp['d_mean'] = df_sub['mean'].apply(lambda x: abs(x-mean)).reset_index(drop=True)
        df_tmp['d_std'] = df_sub['std'].apply(lambda x: abs(x-std)).reset_index(drop=True)
        df_tmp['d_count'] = df_sub['count'].apply(lambda x: abs(x-count)).reset_index(drop=True)
        df_tmp['d_q25'] = df_sub['q25'].apply(lambda x: abs(x-q25)).reset_index(drop=True)
        df_tmp['d_q50'] = df_sub['q50'].apply(lambda x: abs(x-q50)).reset_index(drop=True)
        df_tmp['d_q75'] = df_sub['q75'].apply(lambda x: abs(x-q75)).reset_index(drop=True)
        df_tmp['d_min'] = df_sub['min'].apply(lambda x: abs(x-min_val)).reset_index(drop=True)
        df_tmp['d_max'] = df_sub['max'].apply(lambda x: abs(x-max_val)).reset_index(drop=True)

        # drop where time series match

        # drop frequency matches below 10^2
        df_tmp = df_tmp[df_tmp['match_score']>10**2]
        if df_tmp.shape[0] == 0:
            print("No match found for:")
            print(f'{ts_name_1} - {fft_type}')
        else:
            df_res = df_res.append(df_tmp)
    
    df_res.reset_index(inplace=True)
    return df_res

### Execute comparison

In [ ]:
df_match_scores = generate_stats(df_test_conv,
                                 df_m4_repo)
#df_match_scores.head()

In [ ]:
df_m4_repo[df_m4_repo["ts_name"]=="H354"].sort_values("type")

In [ ]:
df_test_conv.sort_values("fft_type")

In [ ]:
df_test[0]

In [38]:
foo = pd.read_csv("../data/W184.csv")
cols = ["ts_name", "type", "fhat", "PSD","freq", "freq_apx_idx", "freq_apx"]
foo.columns = cols
foo.head()

,ts_name,type,fhat,PSD,freq,freq_apx_idx,freq_apx
0,W184,fft,1.235193e+06,9.280419e+08,0.000608,0,0.1
1,W184,fft,4.188844e+05,1.067300e+08,0.001217,0,0.1
2,W184,fft,3.003137e+05,5.485908e+07,0.001825,0,0.1
3,W184,fft,3.065486e+05,5.716060e+07,0.002433,0,0.1
4,W184,fft,2.017462e+05,2.475763e+07,0.003041,0,0.1


In [35]:
foo.head()

,W184,fft,3324860.2649999997,6724267507.162936,0.0,0,0.0.1
0,W184,fft,1.235193e+06,9.280419e+08,0.000608,0,0.1
1,W184,fft,4.188844e+05,1.067300e+08,0.001217,0,0.1
2,W184,fft,3.003137e+05,5.485908e+07,0.001825,0,0.1
3,W184,fft,3.065486e+05,5.716060e+07,0.002433,0,0.1
4,W184,fft,2.017462e+05,2.475763e+07,0.003041,0,0.1


In [39]:
foo[foo["type"]=="fft"].sort_values("PSD", ascending=False)[:5]

,ts_name,type,fhat,PSD,freq,freq_apx_idx,freq_apx
0,W184,fft,1.235193e+06,9.280419e+08,0.000608,0,0.100000
1642,W184,fft,1.235193e+06,9.280419e+08,0.999392,103,1.017033
1,W184,fft,4.188844e+05,1.067300e+08,0.001217,0,0.100000
1641,W184,fft,4.188844e+05,1.067300e+08,0.998783,103,1.017033
1639,W184,fft,3.065486e+05,5.716060e+07,0.997567,103,1.017033


In [ ]:
[9.98783455e-01 1.21654501e-03 6.08272506e-04 9.99391727e-01 0.00000000e+00]

In [60]:
def get_freq_test(s: pd.Series, k:int=5) -> List[float]:
    """ compute frequencies for M4 pandas series"""
    df = pd.DataFrame()
    f=np.array(s.iloc[1:].dropna())

    n = f.size
    wdw = np.hamming(n)
    freqs = np.arange(n)/n

    # FFT
    fhat = np.abs(np.fft.fft(f))
    PSD = np.real(fhat * np.conj(fhat) / n)
    df= pd.DataFrame({"freqs": freqs, "fhat": fhat, "PSD": PSD})
    return df

In [61]:
df_test[0]

'W184'

In [62]:
df_foo = get_freq_test(df_test[1:], 5)

In [63]:
df_foo.sort_values("PSD", ascending=False)[:5]

,freqs,fhat,PSD
0,0.000000,3.324330e+06,6.726215e+09
1,0.000609,1.234231e+06,9.271620e+08
1642,0.999391,1.234231e+06,9.271620e+08
1641,0.998783,4.187332e+05,1.067179e+08
2,0.001217,4.187332e+05,1.067179e+08
